In [4]:
import os
from dotenv import load_dotenv
load_dotenv()
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")
os.environ['KMP_DUPLICATE_LIB_OK'] = "TRUE"

In [5]:
## Data Ingestion--From the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader

In [6]:
loader=WebBaseLoader("https://docs.smith.langchain.com/tutorials/Administrators/manage_spend")
loader

In [7]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/tutorials/Administrators/manage_spend', 'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='\n\n\n\n\nOptimize tracing spend on LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\nSkip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsAdministratorsOptimize tracing spend on LangSmithDevelopersHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudTutorialsAdministratorsOptimize tracing spend on LangSmithOn this pageOptimize tracing spend on LangSmithRecommended ReadingBefore diving into this content, it might be helpful to read the following:Data Retention Conceptual DocsUsage Limiting Conceptual DocsnoteSome of the features mentioned in this guide are not currently available in Enterprise plan due to its\ncustom nature of billing. If you are on Enterprise plan a

In [8]:
docs[0].metadata

{'source': 'https://docs.smith.langchain.com/tutorials/Administrators/manage_spend',
 'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith',
 'description': 'Before diving into this content, it might be helpful to read the following:',
 'language': 'en'}

In [9]:
docs[0].page_content

'\n\n\n\n\nOptimize tracing spend on LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\nSkip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsAdministratorsOptimize tracing spend on LangSmithDevelopersHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudTutorialsAdministratorsOptimize tracing spend on LangSmithOn this pageOptimize tracing spend on LangSmithRecommended ReadingBefore diving into this content, it might be helpful to read the following:Data Retention Conceptual DocsUsage Limiting Conceptual DocsnoteSome of the features mentioned in this guide are not currently available in Enterprise plan due to its\ncustom nature of billing. If you are on Enterprise plan and have questions about cost optimization,\nplease reach out to your sales rep or support@langchain.dev.This tutorial walks through optimizing your spend on LangSmith. In it, we will learn how to optimize existing spend\nand prevent future overspend on a realistic real-world example. We will us

In [10]:
### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)
documents

[Document(metadata={'source': 'https://docs.smith.langchain.com/tutorials/Administrators/manage_spend', 'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/tutorials/Administrators/manage_spend', 'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsAdministratorsOptimize tracing spend on LangSmithDevelopersHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudTutorialsAdministratorsOptimize tracing spend on LangSmithOn this pageOptimize tracing spend on LangSmithRecommended ReadingBefore diving into this co

In [11]:
len(documents)

18

In [12]:
from langchain_community.embeddings import OllamaEmbeddings
embeddings=OllamaEmbeddings(model="gemma2:2b")
embeddings

OllamaEmbeddings(base_url='http://localhost:11434', model='gemma2:2b', embed_instruction='passage: ', query_instruction='query: ', mirostat=None, mirostat_eta=None, mirostat_tau=None, num_ctx=None, num_gpu=None, num_thread=None, repeat_last_n=None, repeat_penalty=None, temperature=None, stop=None, tfs_z=None, top_k=None, top_p=None, show_progress=False, headers=None, model_kwargs=None)

In [13]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)
vectorstoredb

In [14]:
## Query From a vector db
query="LangSmith has two usage limits: total traces and extended"
result=vectorstoredb.similarity_search(query)
result[0].page_content

'meaning we log around 100,000-130,000 traces per dayExpected Growth in Load: We expect to double in size in the near future.From these assumptions, we can do a quick back-of-the-envelope calculation to get a good limit of:limit = current_load_per_day * expected_growth * days/month      = 130,000 * 2 * 30      = 7,800,000 traces / monthWe click on the edit icon on the right side of the table for our Prod row, and can enter this limit as follows:noteWhen set without the extended data retention traces limit, the maximum cost estimator assumes that all traces are using extended data retention.Cutting maximum spend with an extended data retention limit\u200bIf we are not a big enterprise, we may shudder at the ~$40k per month bill.We saw from Optimization 1 that the easiest way to cut cost was through managing data retention.\nThe same can be said for limits. If we only want to keep ~10% of traces to be around more than 14 days, we can set a limit on the maximum'

In [15]:
from transformers import pipeline
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

c:\Users\ACER 5\Desktop\ASHWINI KAKDE\Langchain\langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
# Initialize the Hugging Face pipeline for question-answering
llm = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

In [20]:
# Create prompt for the document chain
prompt = ChatPromptTemplate.from_template(
    """Answer the following question based only on the provided context:
    <context>{context}</context>"""
)

In [ ]:
def create_stuff_documents_chain(llm, prompt):
    def chain(inputs):
        context = inputs["context"]
        question = inputs["question"]
        answer = llm(question=question, context=context)
        return answer["answer"]
    return chain

# Create document chain
document_chain = create_stuff_documents_chain(llm, prompt)

In [41]:
from langchain.docstore.document import Document
def ask_question(document_chain, documents_string, question):
    # Parse the string to extract Document objects
    import ast
    import re
    
    try:
        # Extract page_content using regex
        page_contents = re.findall(r"page_content='([^']*)'", documents_string)
        
        # Create Document objects
        doc_list = [Document(page_content=content) for content in page_contents]
        
        # Create context string
        context = " ".join(doc.page_content for doc in doc_list)
        
        # Create inputs dictionary
        inputs = {"context": context, "question": question}
        
        # Use the chain
        response = document_chain(inputs)
        return response
        
    except Exception as e:
        return f"Error processing documents: {str(e)}"

In [ ]:
# Example usage
question = "How can administrators manage spend?"
answer = ask_question(document_chain, docs, question)
print(answer)